# Segmenting and Clustering Neighborhoods in Toronto
## Dawid Adrian Bieniek
Assignment for Capstone Project Course.<br>
I'm putting all my work in a single file.

## As GitHub doesn't allow custom JavaScript rendering, use this nbviewer link for this Notebook to view Folium maps etc!:
[https://nbviewer.jupyter.org/github/mrbleaney/Coursera_Capstone/blob/main/TorontoBoroughs.ipynb]

# Task 1

Import libraries:

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

from bs4 import BeautifulSoup # for web scraping

print('Libraries imported.')

Libraries imported.


Get Toronto Postal Codes, Boroughs and Neighborhoods from a Wikipedia page and format them accordingly.

If there is more than one neighborhood in a postal code area, keep them comma separated in a single df cell.
If a borough is "Not Assigned", skip postal code. 
All boroughs have neighborhoods assigned, so I'm not checking for "Not Assigned" neighborhoods:

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
soup = BeautifulSoup(requests.get(url).text, 'html5lib')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df = pd.DataFrame(table_contents)

df['Borough']= df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [4]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [5]:
df.shape

(103, 3)

# TASK 2
Here is a commented, working geocoder code for getting neighborhoods' coordinates based on their postal code, although I'll be using an already generated .csv file later on.

```python
import geocoder # import geocoder

#Initialize empty lists for df columns
latitudes = []
longitudes = []

for postal_code in df.PostalCode.values:

    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitudes.append(lat_lng_coords[0])
    longitudes.append(lat_lng_coords[1])
df['Latitude'] = latitudes
df['Longitude'] = longitudes
print('Done!') 
```

Read new data to pandas dataframe and check its header:

In [6]:
df2 = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
df2.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df2.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge both dataframes:

In [7]:
df_merged = pd.merge(df, df2, how='left', on='PostalCode')
df_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


# TASK 3
Explore and cluster the neighborhoods in Toronto. I'll be exploring only neighborhoods with "Toronto" in the borough name.

## Introduction
I'll be clustering selected neighborhoods in Toronto using the following methodology:
* get "center" coordinates of a neighborhood, based on postal code are and geocoder library
* for each neighborhood, get all venues that are at most 600 m from the "center" of a neighborhood, using Foursquare API
* group venues by category
* for each neighborhood, get mean occurance of each venue category
* use that data for clustering neighborhoods using k-means algorithm, in my case k=7
* visualize clustering and interpret it, looking at most common venue categories in particular neighborhoods

## Prepare data

Choose only boroughs containing "Toronto" in their names:

In [8]:
toronto_data = df_merged[df_merged['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


Create a map

In [9]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.651070, -79.347015], zoom_start=12)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Get venues data from Foursquare API

Here is a code for getting venue categories from Foursquare API by coordinates. <br>
It's not working however, as I can't share my secret credentials, so I'll be using a .csv generated locally by this code, later on:

```python
# Credential data
CLIENT_ID = '' 
CLIENT_SECRET = '' 
VERSION = '20180605' 
LIMIT = 100 

def getNearbyVenues(names, latitudes, longitudes, radius=600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=toronto_data['PostalCode'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])

# Save output
toronto_venues.to_csv('toronto_venues.csv', index=False)
print('Success!')
```

In [24]:
toronto_venues = pd.read_csv('toronto_venues.csv')
print(toronto_venues.shape)
toronto_venues.head()

(2034, 7)


,PostalCode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,M5A,43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park


Prepare data for clustering (one hot encoding and mean venue occurence by postal code area):

In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postal code column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 

# move postal code column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# get mean venue occurence by postal code
toronto_grouped = toronto_onehot.groupby('PostalCode').mean().reset_index()
toronto_grouped.head()

,PostalCode,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.100000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.05,0.000000,0.000000,0

## Explore data

Get top 10 most common venues by postal code area:

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Pub,Bakery,Thai Restaurant,Ice Cream Shop,Indian Restaurant,Gastropub,French Restaurant,Mexican Restaurant,Neighborhood,Electronics Store
1,M4J,Gastropub,Restaurant,Electronics Store,Park,Café,Beer Bar,Dim Sum Restaurant,Coffee Shop,Pizza Place,Hostel
2,M4K,Greek Restaurant,Coffee Shop,Pub,Italian Restaurant,Café,Ice Cream Shop,Bank,Grocery Store,Furniture / Home Store,Cosmetics Shop
3,M4L,Restaurant,Pet Store,Fast Food Restaurant,Sandwich Place,Gym,Park,Dry Cleaner,Furniture / Home Store,Coffee Shop,Liquor Store
4,M4M,Café,Coffee Shop,Bar,Bakery,Diner,American Restaurant,Italian Restaurant,Gastropub,Brewery,Vietnamese Restaurant


Fit K-means clustering model to the data, merge dataframes and add cluster labels:

In [27]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=42).fit(toronto_grouped_clustering)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('PostalCode'), on='PostalCode')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,Coffee Shop,Park,Theater,Bakery,Restaurant,Café,Pub,Breakfast Spot,Gastropub,Mexican Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,3,Coffee Shop,Clothing Store,Hotel,Bubble Tea Shop,Burger Joint,Tea Room,Italian Restaurant,Japanese Restaurant,Restaurant,Cosmetics Shop
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Coffee Shop,Café,Seafood Restaurant,Gastropub,Cosmetics Shop,Theater,American Restaurant,Italian Restaurant,Clothing Store,Cocktail Bar
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Pub,Bakery,Thai Restaurant,Ice Cream Shop,Indian Restaurant,Gastropub,French Restaurant,Mexican Restaurant,Neighborhood,Electronics Store
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Coffee Shop,Cocktail Bar,Seafood Restaurant,Japanese Restaurant,Restaurant,Bakery,Hotel,Sporting Goods Shop,Movie Theater,Farmers Market


Create a colored map for visualizing clusters:

In [28]:
# create map
map_clusters = folium.Map(location=[43.651070, -79.347015], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Conclusions

As we can see from the map and the following tables: 
* Cluster 3 - old town, food-oriented area, heavily dominated by caffes and coffeshops, seconded by restaurants
* Cluster 1 - surrounding cluster 3, very similar to it, although much more diverse and less food-oriented, with much more pubs/bars and occasional book stores, gift shops or pharmacies
* Cluster 2 - green area, dominated by parks, playgrounds and gyms
* Cluster 5 - island/airport area
* Clusters 4 & 6 - very diverse, without obvious character
* Cluster 0 - bussiness service area, the furthest from city-center

### Cluster 0

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Lawrence Park,Bus Line,Park,Swim School,Adult Boutique,Noodle House,Nightclub,New American Restaurant,Neighborhood,Music Venue,Museum


### Cluster 1

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,The Beaches,Pub,Bakery,Thai Restaurant,Ice Cream Shop,Indian Restaurant,Gastropub,French Restaurant,Mexican Restaurant,Neighborhood,Electronics Store
6,Christie,Grocery Store,Café,Park,Baby Store,Athletics & Sports,Candy Store,Italian Restaurant,Coffee Shop,Nightclub,Playground
8,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Park,Convenience Store,Portuguese Restaurant,Discount Store,Coffee Shop,Pet Store,Café,Music Venue
11,"Little Portugal, Trinity",Bar,Coffee Shop,Restaurant,Japanese Restaurant,Café,Theater,Cocktail Bar,Asian Restaurant,Men's Store,Vegetarian / Vegan Restaurant
12,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Pub,Italian Restaurant,Café,Ice Cream Shop,Bank,Grocery Store,Furniture / Home Store,Cosmetics Shop
14,"Brockton, Parkdale Village, Exhibition Place",Café,Gift Shop,Breakfast Spot,Restaurant,Sandwich Place,Coffee Shop,Bar,Neighborhood,Supermarket,Furniture / Home Store
15,"India Bazaar, The Beaches West",Restaurant,Pet Store,Fast Food Restaurant,Sandwich Place,Gym,Park,Dry Cleaner,Furniture / Home Store,Coffee Shop,Liquor Store
17,Studio District,Café,Coffee Shop,Bar,Bakery,Diner,American Restaurant,Italian Restaurant,Gastropub,Brewery,Vietnamese Restaurant
20,Davisville North,Pizza Place,Brewery,Breakfast Spot,Café,Sandwich Place,Park,Sushi Restaurant,Greek Restaurant,Pharmacy,Gym
22,"High Park, The Junction South",Café,Grocery Store,Antique Shop,Italian Restaurant,Mexican Restaurant,Thai Restaurant,Flea Market,Fried Chicken Joint,Storage Facility,Music Venue


### Cluster 2

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,"Moore Park, Summerhill East",Park,Tennis Court,Gym,Adult Boutique,Museum,Nightclub,New American Restaurant,Neighborhood,Music Venue,Movie Theater
33,Rosedale,Park,Playground,Trail,Adult Boutique,Music Venue,Noodle House,Nightclub,New American Restaurant,Neighborhood,Museum


### Cluster 3

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Park,Theater,Bakery,Restaurant,Café,Pub,Breakfast Spot,Gastropub,Mexican Restaurant
1,"Garden District, Ryerson",Coffee Shop,Clothing Store,Hotel,Bubble Tea Shop,Burger Joint,Tea Room,Italian Restaurant,Japanese Restaurant,Restaurant,Cosmetics Shop
2,St. James Town,Coffee Shop,Café,Seafood Restaurant,Gastropub,Cosmetics Shop,Theater,American Restaurant,Italian Restaurant,Clothing Store,Cocktail Bar
4,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Japanese Restaurant,Restaurant,Bakery,Hotel,Sporting Goods Shop,Movie Theater,Farmers Market
5,Central Bay Street,Coffee Shop,Café,Bubble Tea Shop,Thai Restaurant,Burger Joint,Sandwich Place,Italian Restaurant,Japanese Restaurant,French Restaurant,Sushi Restaurant
7,"Richmond, Adelaide, King",Café,Coffee Shop,Restaurant,Gym,Hotel,Theater,Sushi Restaurant,Thai Restaurant,Japanese Restaurant,American Restaurant
9,The Danforth East,Gastropub,Restaurant,Electronics Store,Park,Café,Beer Bar,Dim Sum Restaurant,Coffee Shop,Pizza Place,Hostel
10,"Harbourfront East, Union Station, Toronto Islands",Coffee Shop,Hotel,Café,Brewery,Boat or Ferry,Park,Sandwich Place,Sushi Restaurant,Steakhouse,Chinese Restaurant
13,"Toronto Dominion Centre, Design Exchange",Coffee Shop,Hotel,Café,Restaurant,American Restaurant,Gym,Japanese Restaurant,Concert Hall,Seafood Restaurant,Vegetarian / Vegan Restaurant
16,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Japanese Restaurant,Gym,Seafood Restaurant,Deli / Bodega,American Restaurant,Concert Hall


### Cluster 4

In [33]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,Business Service,Home Service,Playground,Garden,Music Venue,Adult Boutique,Office,Noodle House,Nightclub,New American Restaurant


### Cluster 5

In [34]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Boutique,Harbor / Marina,Boat or Ferry,Plane,Coffee Shop,Rental Car Location,Bar


### Cluster 6

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Forest Hill North & West,Bus Line,Trail,Sushi Restaurant,Jewelry Store,Music Venue,Noodle House,Nightclub,New American Restaurant,Neighborhood,Adult Boutique
